# Ensemble Vote Classifier

Agora que desenvolvemos alguns modelos relativamente eficazes vamos tentar combinar as suas previsões de modo a obtermos um modelo ainda mais eficaz!

In [196]:
from sklearn.datasets import load_iris
from mlxtend.classifier import EnsembleVoteClassifier
from mlxtend.feature_selection import ColumnSelector
from sklearn.pipeline import make_pipeline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from copy import deepcopy
from random import randint

train = pd.read_csv('prep_train.csv')


avg = 0
iss = []
while len(iss) < 500:
    r = randint(0,1000)
    if r not in iss:        
        X_train, X_test, y_train, y_test = train_test_split(train.drop(['Absent'],axis=1), 
                                                            train['Absent'], test_size=0.30, 
                                                            random_state=r)

        pipe1 = make_pipeline(ColumnSelector(cols=['Reason for absence',  'Age', 'Work load Average/day ']), GaussianNB(priors=None, var_smoothing=1e-09))
        pipe2 = make_pipeline(ColumnSelector(cols=['Reason for absence', 'Month of absence', 'Seasons', 'Distance from Residence to Work', 'Service time', 'Age', 'Work load Average/day ', 'Disciplinary failure', 'Body mass index']), RandomForestClassifier(bootstrap=False, criterion='gini', max_depth=20, max_features=None, min_impurity_decrease=0.001, min_samples_split=16, n_estimators=20, oob_score=False))
        pipe3 = make_pipeline(ColumnSelector(cols=['Reason for absence', 'Disciplinary failure']), LogisticRegression(C=0.1, dual=False, fit_intercept=True, intercept_scaling=0.1, l1_ratio=0.1, max_iter=10, multi_class= 'ovr', penalty='l2', solver='liblinear', tol=0.1))
        pipe4 = make_pipeline(ColumnSelector(cols=['Reason for absence', 'Month of absence', 'Seasons', 'Distance from Residence to Work', 'Service time', 'Age', 'Work load Average/day ', 'Disciplinary failure', 'Body mass index']), AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), learning_rate=0.5, n_estimators=100))
        pipe5 = make_pipeline(ColumnSelector(cols=['Reason for absence']), GradientBoostingClassifier(learning_rate=0.01, loss='deviance', max_depth=3, max_features='auto', min_impurity_decrease=0.1, min_samples_split=2, n_estimators=100))
        pipe6 = make_pipeline(ColumnSelector(cols=['Disciplinary failure']), LinearDiscriminantAnalysis(solver='lsqr', shrinkage=None))
 

        eclf = EnsembleVoteClassifier(clfs=[pipe2, pipe6,pipe6, pipe1])
        
        
        eclf.fit(X_train,y_train)
        predictions = eclf.predict(X_test)
        avg = (avg*len(iss) + accuracy_score(y_test,predictions))/float(len(iss) + 1)
        iss.append(r)
print(avg)


0.8606533333333344


In [200]:
X_train, X_test, y_train, y_test = train_test_split(train.drop(['Absent'],axis=1), 
                                                    train['Absent'], test_size=0.3, 
                                                    random_state=randint(0,1000))

pipe1 = make_pipeline(ColumnSelector(cols=['Reason for absence',  'Age', 'Work load Average/day ']), GaussianNB(priors=None, var_smoothing=1e-09))
pipe2 = make_pipeline(ColumnSelector(cols=['Reason for absence', 'Month of absence', 'Seasons', 'Distance from Residence to Work', 'Service time', 'Age', 'Work load Average/day ', 'Disciplinary failure', 'Body mass index']), RandomForestClassifier(bootstrap=False, criterion='gini', max_depth=20, max_features=None, min_impurity_decrease=0.001, min_samples_split=16, n_estimators=20, oob_score=False))
pipe3 = make_pipeline(ColumnSelector(cols=['Reason for absence', 'Disciplinary failure']), LogisticRegression(C=0.1, dual=False, fit_intercept=True, intercept_scaling=0.1, l1_ratio=0.1, max_iter=10, multi_class= 'ovr', penalty='l2', solver='liblinear', tol=0.1))
pipe4 = make_pipeline(ColumnSelector(cols=['Reason for absence', 'Month of absence', 'Seasons', 'Distance from Residence to Work', 'Service time', 'Age', 'Work load Average/day ', 'Disciplinary failure', 'Body mass index']), AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), learning_rate=0.5, n_estimators=100))
pipe5 = make_pipeline(ColumnSelector(cols=['Reason for absence']), GradientBoostingClassifier(learning_rate=0.01, loss='deviance', max_depth=3, max_features='auto', min_impurity_decrease=0.1, min_samples_split=2, n_estimators=100))
pipe6 = make_pipeline(ColumnSelector(cols=['Disciplinary failure']), LinearDiscriminantAnalysis(solver='lsqr', shrinkage=None))


eclf = EnsembleVoteClassifier(clfs=[pipe2, pipe6,pipe6, pipe1])


eclf.fit(X_train,y_train)
predictions = eclf.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(accuracy_score(y_test,predictions))

[[ 11  21]
 [  0 118]]
0.86
